<a href="https://colab.research.google.com/github/LCaravaggio/scrapers/blob/master/Nueva_P%C3%A1gina_de_Coto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%capture
!pip install -U seleniumbase
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo dpkg -i google-chrome-stable_current_amd64.deb
!sudo apt-get install -f

from seleniumbase import Driver
from seleniumbase import page_actions

driver = Driver(uc=True)
driver.get("https://api.cotodigital.com.ar/sitios/cdigi/productos/pate-foie-swift-90g/_/R-00002644-00002644-200")

In [10]:
from selenium.webdriver.common.by import By
element = driver.find_element(By.XPATH, '/html/body/app-root/app-main/app-layout-render/dynamic-loader/general-slot/dynamic-loader/app-product-landing-page/section/div/div/dynamic-loader/app-product-detail/div/aside/div/article/div[2]/div[4]')

print(element.text)

Precio contado : $ 838,99


In [11]:
element = driver.find_element(By.XPATH, '/html/body/app-root/app-main/app-layout-render/dynamic-loader/general-slot/dynamic-loader/app-product-landing-page/section/div/div/dynamic-loader/app-product-detail/div/div[2]/div[1]/p')

print(element.text)

Pate Foie SWIFT 90g
